## 11.1 그래디언트 소실과 폭주 

### 11.1.1 글로럿과 He 초기화

파이썬은 기본적으로 글로럿 초기화를 사용한다고 한다. 만약 바꾸고 싶으면
```{.python}
keras.layers.Dense(10, activation = 'relu', kernel_initializer = 'he_normal')
```
로 해주면 된다

### 11.1.2 수렴하지 않는 활성화 함수

- LeakyReLU

이렇게 렐루 층을 만들고 적용하려는 층 뒤에 추가하면 된다.

``` {.python}
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer = 'he_normal',
    keras.layers.LeakyReLU(alpha = 0.2),
    [...]
    ])
```    

얘처럼 alpha = 0.2로 두면 PReLU가 된다고 함

- SELU

활성화함수를 바꾸고 커널초기화를 르쿤으로 바꾸면 된다고 함

```{.python}layer = keras.layer.Dense(10, activation = 'selu', kernel_initializer = 'lecun_normal')```

### 11.1.3 Batch Normalization 

아래와 같이 layers로 배치 정규화 층을 만들어주면 된다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import pydot

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
    
])

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [5]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [6]:
model.layers[1]

## 11.2 Transfer Learning 

### 11.2.1 케라스를 사용한 전이학습

#model 불러오기
```{.python}
model_a = keras.models.load_model("my_keras_model.h5")
```

#model 불러서 넣기
```{.python}
model_b_on_a = keras.models.Sequential(model_a.layers[:-1])
model_b_on_a.add(keras.layers.Dense(1, activation = 'sigmoid'))
```

model_b_on_a 를 훈련하면 model_a도 영향을 받는다고 한다. 그러므로 층을 재사용 하기 전에 model_a를 복제해야한다

#clone만들기
```{.python}
model_a_clone = keras.models.clone_model(model_a)
model_a_clone.set_weights(model_a.get_weights())
```

## 11.3 Optimizer

### 11.3.1 Momentum

```{.python}
optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9)
```

모멘텀 파라미터인 베타가 추가되긴 하지만, 보통 0.9에서 좋은 성능을 낸다고 한다.



### 11.3.2 네스테로프 가속 경사

```{.python}
optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, nesterov = True)
```


### 11.3.3 AdaGrad

이는 신경망에서 사용하면 안된다고 한다. 학습률이 너무 감소되어 global minima에 도달하기 전에 멈춰버리기 때문

### 11.3.4 RMSProp

```{.python}
optimizer = keras.optimizers.RMSprop(lr = 0.001, rho = 0.9)
```


### 11.3.5 Adam과 변종들

```{.python}
optimizer = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
```
beta1은 모멘텀 감쇠 파라미터, beta2는 스케일 감쇠 파라미터라고 한다.

### 11.3.6 학습률 스케쥴링

- 거듭제곱 기반 스케쥴링

In [5]:
optimizer = keras.optimizers.SGD(lr = 0.01, decay = 1e-4)

- 지수 기반 스케쥴링

In [6]:
def exponential_decay_fn(epoch) : 
    return 0.01 * 0.1 ** (epoch/20)

In [7]:
def exponential_decay(lr0, s) : 
    def exponential_decay_fn(epoch) : 
        return lr0 * 0.1**(epoch/s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0 = 0.01, s = 20)

- call back 생성

In [8]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(x_train_scaled, y_train, callbacks = [lr_scheduler])

NameError: name 'x_train_scaled' is not defined

- 성능 기반 스케쥴링

In [ ]:
def piecewise_constant_fn(epoch) : 
    if epoch <5 : 
        return 0.01
    elif epoch <15 : 
        return 0.005
    else :
        return 0.001
    
    

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5)

In [9]:
s = 20 * len(x_train) // 32
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

NameError: name 'x_train' is not defined

## 11.4 Regularization

### 11.4.1 L1 & L2

In [10]:
#l2
layer = keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal'
                          , kernel_regularizer = keras.regularizers.l2(0.01))
#l2 & l1
layer = keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal'
                          , kernel_regularizer = keras.regularizers.l1_l2(0.01))

보통 모든 은닉층에 동일한 활성화함수, 동일한 초기화전략, 동일한 규제를 사용하기 때문에 하나로 만들어주는 것이 좋다고 한다.

- 코드 리팩터링

In [11]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, 
                          activation = 'elu'
                          ,kernel_initializer = 'he_normal'
                          , kernel_regularizer = keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28])
    ,RegularizedDense(300)
    ,RegularizedDense(100)
    
    #출력층 생성
    ,RegularizedDense(10, activation = 'softmax'
                     ,kernel_initializer = 'glorot_uniform')
    
])

> partial함수 : 하나이상의 인수가 채워진 함수의 새 버전을 만들기 위해 사용된다. 저기선 나머지는 고정돼 있고, units만 새로이 바꿔주는 것

### 11.4.2 Dropout

드롭아웃으로 훈련된 뉴런은 이웃한 뉴런에 의존할 수 없기 때문에 최적의 결과를 내기 위해 좋은 성능을 가지는 방향으로 학습힌다.

In [7]:
model = keras.models.Sequential([
    #입력층
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.Dropout(rate = 0.2),
    
    #은닉층 1
    keras.layers.Dense(300, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate = 0.2),
    
    #은닉층2
    keras.layers.Dense(100, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate = 0.2),
    
     #출력층
    keras.layers.Dense(100, activation = 'softmax')
])

### 11.4.3 Monte-Carlo Dropout

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

fashion_mnist = keras.datasets.fashion_mnist

#데이터 get
(x_train_full, y_train_full), (x_holdout, y_holdout) = fashion_mnist.load_data()

#훈련 데이터 - 검증 데이터 분리
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size = 0.3)

In [9]:
%whos

Variable           Type          Data/Info
------------------------------------------
fashion_mnist      module        <module 'tensorflow.keras<...>hion_mnist\\__init__.py'>
keras              module        <module 'tensorflow.keras<...>low\\keras\\__init__.py'>
model              Sequential    <tensorflow.python.keras.<...>ct at 0x000001E160A3C3A0>
np                 module        <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd                 module        <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
pydot              module        <module 'pydot' from 'C:\<...>site-packages\\pydot.py'>
tf                 module        <module 'tensorflow' from<...>tensorflow\\__init__.py'>
train_test_split   function      <function train_test_split at 0x000001E16ADFA940>
x_holdout          ndarray       10000x28x28: 7840000 elems, type `uint8`, 7840000 bytes (7.476806640625 Mb)
x_train            ndarray       42000x28x28: 32928000 elems, type `uint8`, 32928000 bytes (31.4025

- 기본 모델 구축

In [10]:
#------------------기본 모델 구축
#모델 구축 
model_base = keras.models.Sequential([
    #입력층
    keras.layers.Flatten(input_shape = [28,28]),
    #은닉층1
    keras.layers.Dense(300, activation = 'elu', kernel_initializer='he_normal'),
    #은닉층2
    keras.layers.Dense(100, activation = 'elu', kernel_initializer='he_normal'),
    #출력층
    keras.layers.Dense(10, activation = 'softmax')
])

#모델 컴파일
model_base.compile(loss = 'sparse_categorical_crossentropy', 
             optimizer = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999), #Adam
              metrics = ['accuracy'] #훈련과 평가 시에 정확도를 측정하기 위해 accuracy를 사용!
             )

#early_stopping 지정
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights=True)

#모델 적합
history = model_base.fit(x = x_train, y = y_train, epochs = 100 #30번 반복?
                   , validation_data = (x_valid, y_valid) #이렇게 검증 셋을 지정해줄 수 있음
                   ,verbose = 0, callbacks = [early_stopping_cb]) # 더이상 결과가 창에 뜨지 않는다!

#validation에 대한 성능 검증
model_base.evaluate(x_valid, y_valid)

563/563 [==============================] - 1s 1ms/step - loss: 0.3734 - accuracy: 0.8753


[0.3733670115470886, 0.875333309173584]

In [11]:
np.round(model_base.predict(x_holdout[:1]),2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],
      dtype=float32)

> 이 모델은 새로운 데이터가 거의 앵클부츠(91%로의 확률로)에 해당한다고 평가한다.

- Monte Carlo Drop

In [14]:
#Monte-Carlo 정의
class MCDropout(keras.layers.Dropout) : 
    def call(self, inputs) : 
         return super().call(inputs, training = True)
        
        
#모델 구축 
model_mc = keras.models.Sequential([
    #입력층
    keras.layers.Flatten(input_shape = [28,28]),
    MCDropout(rate = 0.2),
    #은닉층1
    keras.layers.Dense(300, activation = 'elu', kernel_initializer='he_normal'),
    MCDropout(rate = 0.2),
    #은닉층2
    keras.layers.Dense(100, activation = 'elu', kernel_initializer='he_normal'),
    MCDropout(rate = 0.2),
    #출력층
    keras.layers.Dense(10, activation = 'softmax')
])

#모델 컴파일
model_mc.compile(loss = 'sparse_categorical_crossentropy', 
             optimizer = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999), #Adam
              metrics = ['accuracy'] #훈련과 평가 시에 정확도를 측정하기 위해 accuracy를 사용!
             )

#early_stopping 지정
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights=True)

#모델 적합
history = model_mc.fit(x = x_train, y = y_train, epochs = 100 #30번 반복?
                   , validation_data = (x_valid, y_valid) #이렇게 검증 셋을 지정해줄 수 있음
                   ,verbose = 0, callbacks = [early_stopping_cb]) # 더이상 결과가 창에 뜨지 않는다!

#validation에 대한 성능 검증
model_mc.evaluate(x_valid, y_valid)

563/563 [==============================] - 1s 2ms/step - loss: 0.6076 - accuracy: 0.8021


[0.6076083183288574, 0.8020555377006531]

In [15]:
np.round(model_mc.predict(x_holdout[:1]),2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],
      dtype=float32)